# Create sales report using SQL and Pandas

In [2]:
from datetime import date
from pathlib import Path
import sqlite3

import pandas as pd  
import plotly.express as px  
# from fpdf import FPDF  

## 1. Define path and chart types

In [21]:
db_path = './sales.db'
# output_dir = current_dir / "output"
# Create the output directory and its parent directory if they do not exist
# output_dir.mkdir(parents=True, exist_ok=True)
# Define the plotly template. Some other examples:
# "plotly", "ggplot2", "seaborn", "simple_white", "plotly_dark", "plotly_white", ..
plotly_template = "presentation"

## 2. Total sales by month?

In [4]:
# Create a connection to the database
conn = sqlite3.connect(db_path)

In [5]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT sale_date, SUM(total_price) as total_sales
FROM sales
GROUP BY sale_date
ORDER BY sale_date ASC
'''
df = pd.read_sql_query(query, conn)

In [6]:
df

,sale_date,total_sales
0,2024-01-01,1485
1,2024-01-02,1915
2,2024-01-03,400
3,2024-01-04,320
4,2024-01-05,1125
...,...,...
334,2024-12-26,1060
335,2024-12-27,1275
336,2024-12-28,1060
337,2024-12-29,1065


In [7]:
# Check the data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sale_date    339 non-null    object
 1   total_sales  339 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 4.0+ KB


In [8]:
# Convert sale_date to datetime
df['sale_date'] = pd.to_datetime(df['sale_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   sale_date    339 non-null    datetime64[ns]
 1   total_sales  339 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.4 KB


In [10]:
# Set the sale_date column as the index
df = df.set_index('sale_date')
df.head()

,total_sales
sale_date,
2024-01-01,1485
2024-01-02,1915
2024-01-03,400


In [11]:
# Resample the data to a monthly frequency and compute the sum
df_monthly = df.resample('M').sum()
df_monthly

,total_sales
sale_date,
2024-01-31,28410
2024-02-29,23465
2024-03-31,17745
2024-04-30,26655
2024-05-31,24040
2024-06-30,20965
2024-07-31,26350
2024-08-31,21970
2024-09-30,21250


In [12]:
# Map the month number to short month name
df_monthly['month_name'] = df_monthly.index.strftime('%b')
df_monthly

,total_sales,month_name
sale_date,,
2024-01-31,28410,Jan
2024-02-29,23465,Feb
2024-03-31,17745,Mar
2024-04-30,26655,Apr
2024-05-31,24040,May
2024-06-30,20965,Jun
2024-07-31,26350,Jul
2024-08-31,21970,Aug
2024-09-30,21250,Sep


In [14]:
# Create the Plotly figure with text parameter
fig = px.bar(df_monthly,
             x='month_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Total Sales by Month',
    xaxis_title='Month',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
# fig.write_image(output_dir / 'monthly_sales.png',
#                 width=1200,
#                 height=400,
#                 scale=4)

## 3. Total sales by product?

In [15]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT p.product_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN products p ON s.product_id = p.product_id
GROUP BY p.product_name
'''
df = pd.read_sql_query(query, conn)

In [16]:
df

,product_name,total_sales
0,Product A,57650
1,Product B,27375
2,Product C,83850
3,Product D,46400
4,Product E,60900


In [17]:
# Create the Plotly figure with text parameter
fig = px.bar(df,
             x='product_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Total Sales by Product',
    xaxis_title='Product',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
# fig.write_image(output_dir / 'product_sales.png',
#                 width=1200,
#                 height=400,
#                 scale=4)

## 4. Top customer by sales

In [18]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT c.first_name || ' ' || c.last_name as customer_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
GROUP BY customer_name
ORDER BY total_sales DESC
LIMIT 10
'''
df = pd.read_sql_query(query, conn)

In [19]:
df

,customer_name,total_sales
0,Isabel Garcia,33305
1,Emily Davis,32135
2,Grace Lee,29830
3,Henry Chen,29175
4,John Doe,28760
5,Jane Doe,28025
6,Bob Smith,26015
7,Frank Wilson,24865
8,Alice Jones,24810
9,David Brown,19255


In [20]:
# Create the Plotly figure
fig = px.bar(df,
             x='customer_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Top Customers by Sales',
    xaxis_title='Customer',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

## 5. Customer segment

In [22]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT 
  customers.customer_id, 
  customers.first_name || ' ' || customers.last_name as customer_name, 
  SUM(sales.total_price) as total_sales,
  CASE 
    WHEN SUM(sales.total_price) > 30000 THEN 'High Value'
    WHEN SUM(sales.total_price) > 26000 THEN 'Medium Value'
    ELSE 'Low Value'
  END as customer_segment
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
ORDER BY total_sales DESC
'''

df = pd.read_sql_query(query, conn)

In [23]:
df

,customer_id,customer_name,total_sales,customer_segment
0,10,Isabel Garcia,33305,High Value
1,6,Emily Davis,32135,High Value
2,8,Grace Lee,29830,Medium Value
3,9,Henry Chen,29175,Medium Value
4,1,John Doe,28760,Medium Value
5,2,Jane Doe,28025,Medium Value
6,3,Bob Smith,26015,Medium Value
7,7,Frank Wilson,24865,Low Value
8,4,Alice Jones,24810,Low Value
9,5,David Brown,19255,Low Value


In [24]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT 
  customers.customer_id, 
  customers.first_name || ' ' || customers.last_name as customer_name, 
  SUM(sales.total_price) as total_sales
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
'''

df = pd.read_sql_query(query, conn)
df

,customer_id,customer_name,total_sales
0,1,John Doe,28760
1,2,Jane Doe,28025
2,3,Bob Smith,26015
3,4,Alice Jones,24810
4,5,David Brown,19255
5,6,Emily Davis,32135
6,7,Frank Wilson,24865
7,8,Grace Lee,29830
8,9,Henry Chen,29175
9,10,Isabel Garcia,33305


In [25]:
# Group the data by customer segment
bins = [0, 26000, 30000, float('inf')]
labels = ['Low Value', 'Medium Value', 'High Value']
df['customer_segment'] = pd.cut(df['total_sales'], bins=bins, labels=labels)

# Order the data by total sales
df = df.sort_values(by='total_sales', ascending=False)
df

,customer_id,customer_name,total_sales,customer_segment
9,10,Isabel Garcia,33305,High Value
5,6,Emily Davis,32135,High Value
7,8,Grace Lee,29830,Medium Value
8,9,Henry Chen,29175,Medium Value
0,1,John Doe,28760,Medium Value
1,2,Jane Doe,28025,Medium Value
2,3,Bob Smith,26015,Medium Value
6,7,Frank Wilson,24865,Low Value
3,4,Alice Jones,24810,Low Value
4,5,David Brown,19255,Low Value


## 6. Create pdf report

In [ ]:
# Define the font color as RGB values (dark gray)
font_color = (64, 64, 64)

# Find all PNG files in the output folder
chart_filenames = [str(chart_path) for chart_path in output_dir.glob("*.png")]

# Create a PDF document and set the page size
pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 24)

# Add the overall page title
title = f"Sales Report as of {date.today().strftime('%m/%d/%Y')}"
pdf.set_text_color(*font_color)
pdf.cell(0, 20, title, align='C', ln=1)

# Add each chart to the PDF document
for chart_filename in chart_filenames:
    pdf.ln(10)  # Add padding at the top of the next chart
    pdf.image(chart_filename, x=None, y=None, w=pdf.w - 20, h=0)

# Save the PDF document to a file on disk
pdf.output(output_dir / "sales_report.pdf", "F")

In [26]:
# Close the connection
conn.close()